In [291]:
import requests
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st

#prev
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.utils.plotting import plot_series

#visualisation
import  plotly.express  as  px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

#pip install streamlit-echarts
#pip install pyecharts


importation données OHLC

In [292]:
#OHLC BTC
url_BTC = "https://api.coingecko.com/api/v3/coins/bitcoin/ohlc?vs_currency=usd&days=1"
headers = {"accept": "application/json"}
response_BTC = requests.get(url_BTC, headers=headers)
BTC_json= response_BTC.json()

#OHCL ETH

url_ETH = "https://api.coingecko.com/api/v3/coins/ethereum/ohlc?vs_currency=usd&days=1"
headers = {"accept": "application/json"}
response_ETH = requests.get(url_ETH, headers=headers)
ETH_json=response_ETH.json()

#OHCL BNB 
url_BNB = "https://api.coingecko.com/api/v3/coins/binancecoin/ohlc?vs_currency=usd&days=1"
headers = {"accept": "application/json"}
response_BNB = requests.get(url_BNB, headers=headers)
BNB_json=response_BNB.json()


importation données 

capitalisation, capitalisation en % et volume d'echange

In [293]:
# total market cap, total volumes, market cap porcentage et markey cap % variation 24h
url = "https://api.coingecko.com/api/v3/global"
headers = {"accept": "application/json"}
response_market_cap = requests.get(url, headers=headers)
market_cap_json = response_market_cap.json()

im_total_market_cap = market_cap_json['data']['total_market_cap']
df_total_market_cap = pd.DataFrame(list(im_total_market_cap.items()), columns=['currency', 'market_cap'])

im_total_volume = market_cap_json['data']['total_volume']
df_total_volume = pd.DataFrame(list(im_total_volume.items()), columns=['currency', 'total_volume'])

im_market_cap_percentage = market_cap_json['data']['market_cap_percentage']
df_market_cap_percentage = pd.DataFrame(list(im_market_cap_percentage.items()), columns=['currency', 'market_cap_percentage'])


API classement coinmarket cap 

In [294]:
market_cap_json

{'data': {'active_cryptocurrencies': 15274,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 1158,
  'total_market_cap': {'btc': 34918196.43282971,
   'eth': 1005390179.9770168,
   'ltc': 35361046403.47276,
   'bch': 6931250908.54157,
   'bnb': 5068823435.140371,
   'eos': 4773038064551.241,
   'xrp': 2847216484004.9287,
   'xlm': 17944348459964.875,
   'link': 222861809061.969,
   'dot': 581822709774.1752,
   'yfi': 501545161.6962473,
   'usd': 3156609046425.286,
   'aed': 11593909366615.438,
   'ars': 3147848389387599.0,
   'aud': 4885635338386.642,
   'bdt': 378356084614646.5,
   'bhd': 1188349668053.4504,
   'bmd': 3156609046425.286,
   'brl': 18298704811675.055,
   'cad': 4451292246816.611,
   'chf': 2802873123464.7773,
   'clp': 3094518546482103.0,
   'cny': 22830174928270.867,
   'czk': 75650976745723.66,
   'dkk': 22342478830598.188,
   'eur': 2993608068485.9775,
   'gbp': 2501672644863.9224,
   'gel': 8633325741973.146,
   'hkd': 24575779730944.06

LT OHLC BTC SOL ETH CSV 

In [295]:
#nettoyage

#BTC
df_OHLC_BTC= pd.DataFrame(BTC_json, columns=["Timestamp", "Open", "High", "Low", "Close"])
df_OHLC_BTC['Timestamp'] = pd.to_datetime(df_OHLC_BTC['Timestamp'], unit='ms')

#ETH
df_OHLC_ETH= pd.DataFrame(ETH_json, columns=["Timestamp", "Open", "High", "Low", "Close"])
df_OHLC_ETH['Timestamp'] = pd.to_datetime(df_OHLC_ETH['Timestamp'], unit='ms')


#BNB
df_OHLC_BNB= pd.DataFrame(BNB_json, columns=["Timestamp", "Open", "High", "Low", "Close"])
df_OHLC_BNB['Timestamp'] = pd.to_datetime(df_OHLC_BNB['Timestamp'], unit='ms')





PREVISION BTC

In [296]:

#Prévision BTC

#préparation données 
open_BTC = df_OHLC_BTC.drop(['High','Low','Close'], axis=1)
open_BTC.set_index("Timestamp", inplace=True)
x_BTC=pd.Series(open_BTC['Open'])

#préparation ech test et horizon test
BTC_train, BTC_test = temporal_train_test_split(x_BTC,test_size= 10)
fh_test = ForecastingHorizon(BTC_test.index, is_relative=False)

#horizon prévision 
fh_dates = pd.date_range(start=x_BTC.index.max(), periods=10, freq='30min')
fh_pred = ForecastingHorizon(fh_dates, is_relative=False)

# utilisation du modele 
model= ExponentialSmoothing(trend="add", seasonal=None)
model_test_BTC= model.fit(BTC_train)
model_pred_BTC=model.fit(x_BTC) 

# prévision et test 

BTC_perf=model_test_BTC.predict(fh_test)
BTC_pred = model_pred_BTC.predict(fh_pred)
#mape
mape_BTC = mean_absolute_percentage_error(BTC_test, BTC_pred)*100

#fusion 
df_BTC_train= pd.DataFrame(BTC_train)
df_BTC_pred=pd.DataFrame(BTC_pred)
BTC_fusion=pd.DataFrame(pd.concat([BTC_train, BTC_test], axis=0))


/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.

/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.



PREVISION ETH

In [297]:
# Prévision eth
#préparation données 
open_ETH = df_OHLC_ETH.drop(['High','Low','Close'], axis=1)
open_ETH.set_index("Timestamp", inplace=True)
x_ETH=pd.Series(open_ETH['Open'])

#préparation ech test et horizon test
ETH_train, ETH_test = temporal_train_test_split(x_ETH,test_size= 10)
fh_test = ForecastingHorizon(ETH_test.index, is_relative=False)

#horizon prévision 
fh_dates = pd.date_range(start=x_ETH.index.max(), periods=10, freq='30min')
fh_pred = ForecastingHorizon(fh_dates, is_relative=False)

# utilisation du modele 
model= ExponentialSmoothing(trend="add", seasonal=None)
model_test_ETH= model.fit(ETH_train)
model_pred_ETH=model.fit(x_ETH) 

# prévision et test 

ETH_perf=model_test_ETH.predict(fh_test)
ETH_pred = model_pred_ETH.predict(fh_pred)
#mape
mape_ETH = mean_absolute_percentage_error(ETH_test, ETH_pred)*100


#fusion
df_ETH_train= pd.DataFrame(ETH_train)
df_ETH_pred = pd.DataFrame(ETH_pred)
ETH_fusion=pd.DataFrame(pd.concat([ETH_train, ETH_test], axis=0))

/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.

/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.



In [298]:
x_ETH.index.max()

Timestamp('2024-11-16 15:00:00')

PREVISION BNB

In [299]:
open_BNB = df_OHLC_BNB.drop(['High','Low','Close'], axis=1)
open_BNB.set_index("Timestamp", inplace=True)
x_BNB=pd.Series(open_BNB['Open'])




#préparation ech test et horizon test
BNB_train, BNB_test = temporal_train_test_split(x_BNB,test_size= 10)
fh_test = ForecastingHorizon(BNB_test.index, is_relative=False)

#horizon prévision 
fh_dates = pd.date_range(start=x_BNB.index.max(), periods=10, freq='30min')
fh_pred = ForecastingHorizon(fh_dates, is_relative=False)

# utilisation du modele 
model= ExponentialSmoothing(trend="add", seasonal=None)
model_test_BNB= model.fit(BNB_train)
model_pred_BNB=model.fit(x_BNB) 

# prévision et test 

BNB_perf=model_test_BNB.predict(fh_test)
BNB_pred = model_pred_BNB.predict(fh_pred)
#mape
mape_BNB = mean_absolute_percentage_error(BNB_test, BNB_pred)*100

#fusion
df_BNB_train= pd.DataFrame(BNB_train)
df_BNB_pred = pd.DataFrame(BNB_pred)
BNB_fusion=pd.DataFrame(pd.concat([BNB_train, BNB_test], axis=0))

/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.

/Users/nicolasmayeur/vscode/data_visu/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency 30min will be used.



In [300]:
type(mape_ETH)


numpy.float64

fonction prévision

# S = Serie avec donnée open_crypto['open']
# name nom crypto


def prevision(S,name):
    #préparation ech test et horizon test
    y_train, y_test = temporal_train_test_split(S,test_size= 10)
    fh_test = ForecastingHorizon(y_test.index, is_relative=False)

    #horizon prévision 
    fh_dates = pd.date_range(start=S.index[-1], periods=10, freq='30min')
    fh_pred = ForecastingHorizon(fh_dates, is_relative=False)

    # utilisation du modele 
    model= ExponentialSmoothing(trend="add", seasonal=None)
    model_test= model.fit(y_train)
    model_pred=model.fit(S) 

    # prévision et test 
    y_pred = model_test.predict(fh_pred)
    y_perf=model_test.predict(fh_test)
    #mape
    mape = mean_absolute_percentage_error(y_test, y_pred)
    

    df_train = {}
    df_pred = {}
    fusion = {}
    mape={}

    mape[name] = pd.DataFrame(mape)
    df_train[name] = pd.DataFrame(y_train, columns=['train'])
    df_pred[name] = pd.DataFrame(y_pred, columns=['prediction'])
    fusion[name] = pd.concat([y_train, y_test], axis=0)

    return df_train[name], df_pred[name], fusion[name], mape[name]



prevision(x_BTC, 'btc')
df_train['btc']

GRAPHIQUES prévision 

In [301]:
def graph_prev(index_fusion,valeur_y_fusion,index_pred,valeur_pred):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=index_fusion.index.values, y=valeur_y_fusion, mode='lines', name='cours', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=index_pred.index.values, y=valeur_pred, mode='lines', name='prévision', line=dict(color='red')))
    fig.update_layout(
            title="Pévisions ",
            xaxis_title="Temps",
            yaxis_title="prix (usd)",
            legend=dict(x=0, y=1),
            hovermode="x unified",
            template="plotly_white"
        )
    fig.show()


graph_prev(BTC_fusion,BTC_fusion['Open'],df_BTC_pred,df_BTC_pred['Open'])
graph_prev(ETH_fusion,ETH_fusion['Open'],df_ETH_pred,df_ETH_pred['Open'])
graph_prev(BNB_fusion,BNB_fusion['Open'],df_BNB_pred,df_BNB_pred['Open'])

Jauges

JAUGE  performance prévision


In [302]:
# app_streamlit_echarts.py

import streamlit as st
from streamlit_echarts import st_echarts
def visua_mape(mape):
    # Configuration de la jauge
    options = {
        "series": [
            {
                "type": "gauge",
                "startAngle": 180,
                "endAngle": 0,
                "center": ["50%", "75%"],
                "radius": "90%",
                "min": 0,
                "max": 20,
                "splitNumber": 4,
                "axisLine": {
                    "lineStyle": {
                        "width": 6,
                        "color": [
                            [0.25, "seagreen"],   # Zone 0-5%
                            [1, "#DC143C"]        # Zone 5-20%
                        ]
                    }
                },
                "pointer": {
                    "length": "70%",
                    "width": 6,
                    "itemStyle": {
                        "color": "black"
                    }
                },
                "axisTick": {
                    "length": 10,
                    "lineStyle": {
                        "color": "auto",
                        "width": 2
                    }
                },
                "splitLine": {
                    "length": 15,
                    "lineStyle": {
                        "color": "auto",
                        "width": 3
                    }
                },
                "axisLabel": {
                    "fontSize": 12,
                    "distance": -40,
                    "formatter": "{value} %"
                },
                "title": {
                    "show": True,
                    "offsetCenter": [0, "-10%"],
                    "fontSize": 16,
                    "color": "#333",
                    "text": "Qualité du modèle de prévision en %"
                },
                "detail": {
                    "fontSize": 20,
                    "offsetCenter": [0, "25%"],
                    "valueAnimation": True,
                    "formatter": "{value} %",
                    "color": "black"
                },
                "data": [
                    {
                        "value": mape,
                        "name": "MAPE"
                    }
                ]
            }
        ]
    }

    # Afficher la jauge dans Streamlit
    st_echarts(options=options, height="400px")

# Assure-toi que ces variables sont bien définies avant d'appeler la fonction
try:
    # Appel des fonctions pour chaque jauge
    st.subheader("MAPE pour BTC")
    visua_mape(mape_BTC)

    st.subheader("MAPE pour ETH")
    visua_mape(mape_ETH)

    st.subheader("MAPE pour BNB")
    visua_mape(mape_BNB)

except IndexError:
    st.error("Erreur : une des valeurs MAPE est non définie ou en dehors des limites.")



2024-11-16 16:56:46.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.354 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.354 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [303]:

def jauge(mape):

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = mape,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': "qualité du modèle de prévision en %"}))

    fig.show()

jauge(mape_BTC)
jauge(mape_ETH)
jauge(mape_BNB)

PAGE ACCUEIL

Volume échange

In [304]:
df_total_volume

,currency,total_volume
0,btc,2.599357e+06
1,eth,7.484258e+07
2,ltc,2.632323e+09
3,bch,5.159715e+08
4,bnb,3.773299e+08
...,...,...
57,xdr,1.775485e+11
58,xag,7.767854e+09
59,xau,9.168769e+07
60,bits,2.599357e+12


In [305]:
filtered_df = df_total_volume[df_total_volume['currency'].isin(['btc', 'eth', 'bnb'])]
filtered_df

,currency,total_volume
0,btc,2.599357e+06
1,eth,7.484258e+07
4,bnb,3.773299e+08


In [306]:
df = px.data.stocks()
fig = px.histogram(filtered_df, x='currency',y='total_volume')
fig.update_layout(bargap=0.2)
fig.show()

In [307]:


url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=7"

headers = {"accept": "application/json"}

response_BTC2 = requests.get(url, headers=headers)
BTC2_json= response_BTC2.json()
df_OHLC_BTC2= pd.DataFrame(BTC2_json, columns=["Timestamp", "price", "Higmarket_cap", "volume"])
df_OHLC_BTC2['Timestamp'] = pd.to_datetime(df_OHLC_BTC2['Timestamp'], unit='ms')

In [308]:
BTC_volume_df=pd.DataFrame(BTC2_json['total_volumes'], columns=['Timestamp','price'])
BTC_volume_df['Timestamp'] = pd.to_datetime(df_OHLC_BTC2['Timestamp'])
BTC_market_cap_df=pd.DataFrame(BTC2_json['market_caps'], columns=['Timestamp','price'])


In [309]:
BTC_volume_df

,Timestamp,price
0,NaT,3.708132e+10
1,NaT,3.750733e+10
2,NaT,3.862192e+10
3,NaT,3.407141e+10
4,NaT,3.403451e+10
...,...,...
164,NaT,7.356461e+10
165,NaT,7.503084e+10
166,NaT,7.432652e+10
167,NaT,6.729969e+10


Graphique prévision animmé 

In [310]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Définir la fonction pour afficher l'animation avec une séquence différée entre les lignes
def graph_prev_2(index_fusion, valeur_y_fusion, index_pred, valeur_pred):
    # S'assurer que les données sont en 1D
    index_fusion = pd.Series(index_fusion).values
    valeur_y_fusion = pd.Series(valeur_y_fusion).values
    index_pred = pd.Series(index_pred).values
    valeur_pred = pd.Series(valeur_pred).values
    
    # Créer des DataFrames pour les données originales et les prévisions
    df_fusion = pd.DataFrame({'Date': index_fusion, 'Valeur': valeur_y_fusion, 'Type': 'cours'})
    df_pred = pd.DataFrame({'Date': index_pred, 'Valeur': valeur_pred, 'Type': 'prévision'})
    
    # Calculer le nombre de frames pour chaque ligne
    frames = []
    max_length_fusion = len(df_fusion)
    max_length_pred = len(df_pred)

    # Créer la figure initiale
    fig = go.Figure()

    # Ajouter les deux lignes initialement vides pour l'animation
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Cours', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Prévision', line=dict(color='red')))

    # Définir les frames pour l'animation
    for i in range(1, max_length_fusion + max_length_pred + 1):
        frame_data = []

        # Afficher les données du cours jusqu'à ce que la série soit complète
        if i <= max_length_fusion:
            frame_data.append(go.Scatter(x=df_fusion['Date'][:i], y=df_fusion['Valeur'][:i], mode='lines', line=dict(color='blue')))
            frame_data.append(go.Scatter(x=[], y=[], mode='lines', line=dict(color='red')))  # Prévision vide avant la fin du cours
        # Commencer à afficher la prévision une fois le cours terminé
        else:
            frame_data.append(go.Scatter(x=df_fusion['Date'], y=df_fusion['Valeur'], mode='lines', line=dict(color='blue')))
            j = i - max_length_fusion  # index pour les données de prévision
            frame_data.append(go.Scatter(x=df_pred['Date'][:j], y=df_pred['Valeur'][:j], mode='lines', line=dict(color='red')))

        # Ajouter le frame
        frames.append(go.Frame(data=frame_data, name=str(i)))

    # Ajouter les frames à la figure
    fig.frames = frames

    # Ajuster les attributs de la figure
    fig.update_layout(
        xaxis_title="Temps",
        yaxis_title="prix (usd)",
        title="Prévisions ",
        hovermode="x unified",
        template="plotly_white",
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
            }]
        }]
        
    )

    st.plotly_chart(fig)

# Exécuter la fonction avec animation
graph_prev_2(BTC_fusion.index, BTC_fusion['Open'], df_BTC_pred.index, df_BTC_pred['Open'])
graph_prev_2(ETH_fusion.index,ETH_fusion['Open'],df_ETH_pred.index,df_ETH_pred['Open'])
graph_prev_2(BNB_fusion.index,BNB_fusion['Open'],df_BNB_pred.index,df_BNB_pred['Open'])


2024-11-16 16:56:46.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

jauge animation

In [311]:
import plotly.graph_objects as go

def jauge(mape):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=mape,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Qualité du modèle de prévision en %"},
        gauge={
            'axis': {'range': [0, 20]},  # Plage de 0 à 100%
            'bar': {'color': "black"},      # Couleur de la barre
            'steps': [
                {'range': [0, 5], 'color': "seagreen"},  # Zone 0-50%
                {'range': [5, 20], 'color': "#DC143C"}      # Zone 50-100%
            ]
        }
    ))

    # Configuration du layout
    fig.update_layout(
        hovermode="x unified",
        template="plotly_white"
    )

    # Afficher la figure
    st.plotly_chart(fig)


jauge(mape_BTC)
jauge(mape_ETH)
jauge(mape_BNB)



2024-11-16 16:56:46.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [312]:
essa = jauge(mape_BTC)

2024-11-16 16:56:46.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-16 16:56:46.967 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


PAGE INDICATEURS en temps réel

GRAPHIQUE bougie avec BB et Volume 

In [313]:

import numpy as np



def calcul_BB(prix, valeurs_passees=10, ecart=5):
    #tableau 1 dim pour calcul
    prix = np.asarray(prix).flatten()
    prix = pd.Series(prix)

    rolling_mean = prix.rolling(valeurs_passees, min_periods=1).mean()
    rolling_std = prix.rolling(valeurs_passees, min_periods=1).std()
    
    # Calcul des bandes de Bollinger
    bande_supp = rolling_mean + ecart * rolling_std
    bande_inf = rolling_mean - ecart * rolling_std
    
    return bande_supp.values, bande_inf.values


# Exemple d'appel à la fonction avec open_BTC
B_supp_BTC, B_inf_BTC = calcul_BB(open_BTC)
B_supp_ETH, B_inf_ETH = calcul_BB(open_ETH)
B_supp_BNB, B_inf_BNB = calcul_BB(open_BNB)







def graph_BB(df, bande_supp, bande_inf,nom ):

    

    fig = go.Figure()
    
    #bougies OHLC
    fig.add_trace(go.Candlestick(
        x=df['Timestamp'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name="OHLC",
        increasing_line_color='green',
        decreasing_line_color= 'red'
        

        
    ))
    
    # BB supp
    fig.add_trace(go.Scatter(
        x=df['Timestamp'],  
        y=bande_supp,  
        mode='lines',
        name='Upper Bollinger Band',
        line=dict(color='red')
        
    ))
    
    #BB inf
    fig.add_trace(go.Scatter(
        x=df['Timestamp'],  
        y=bande_inf,    
        mode='lines',
        name='Lower Bollinger Band',
        line=dict(color='green')
        
    ))
    fig.update_layout(template="plotly_dark",
                      xaxis_title="périodes",
                      yaxis_title="prix en dollars ",
                      title="graphiques OHLC  "+nom)
   
    fig.show()

graph_BB_BTC=graph_BB(df_OHLC_BTC,B_supp_BTC, B_inf_BTC,nom = "du Bitcoin")
graph_BB_ETH=graph_BB(df_OHLC_ETH,B_supp_ETH, B_inf_ETH,nom = "d'Ethereum")
graph_BB_BNB=graph_BB(df_OHLC_BNB,B_supp_BNB, B_inf_BNB, nom= "du Binance Coin")
 

RSI

In [314]:
def RSI_indica(prix, period=10):
    delta = prix.diff()
    delta = delta.dropna()
    up, down = delta.clip(lower=0), delta.clip(upper=0, lower=None)
    ema_up = up.ewm(alpha=1/period, min_periods=period).mean()
    ema_down = down.abs().ewm(alpha=1/period, min_periods=period).mean()
    rs = ema_up / ema_down
    rsi = 100 - 100 / (1 + rs)
    
    return rsi, ema_up, ema_down

RSI_indica(open_BTC['Open'])





(Timestamp
 2024-11-15 16:00:00          NaN
 2024-11-15 16:30:00          NaN
 2024-11-15 17:00:00          NaN
 2024-11-15 17:30:00          NaN
 2024-11-15 18:00:00          NaN
 2024-11-15 18:30:00          NaN
 2024-11-15 19:00:00          NaN
 2024-11-15 19:30:00          NaN
 2024-11-15 20:00:00          NaN
 2024-11-15 20:30:00    71.866464
 2024-11-15 21:00:00    76.457377
 2024-11-15 21:30:00    79.866939
 2024-11-15 22:00:00    78.037367
 2024-11-15 22:30:00    80.972558
 2024-11-15 23:00:00    67.576257
 2024-11-15 23:30:00    67.830957
 2024-11-16 00:00:00    68.193065
 2024-11-16 00:30:00    62.703383
 2024-11-16 01:00:00    58.609310
 2024-11-16 01:30:00    60.446493
 2024-11-16 02:00:00    67.360610
 2024-11-16 02:30:00    62.464645
 2024-11-16 03:00:00    68.384728
 2024-11-16 03:30:00    62.257393
 2024-11-16 04:00:00    60.577895
 2024-11-16 04:30:00    59.844078
 2024-11-16 05:00:00    55.168782
 2024-11-16 05:30:00    57.777493
 2024-11-16 06:00:00    63.118201
 20

In [315]:
def RSI_indica (prix, period=14):
    delta = prix.diff()
    up = delta.where(delta > 0, 0)  
    down = -delta.where(delta < 0, 0)

    ema_up = up.ewm(alpha=1/period, min_periods=period).mean()
    ema_down = down.abs().ewm(alpha=1/period, min_periods=period).mean()
    rs = ema_up / ema_down
    rsi = 100 - 100 / (1 + rs)
    
    return rsi.iloc[period:], ema_up.iloc[period:], ema_down.iloc[period:]



BTC_rsi, BTC_ema_up, BTC_ema_down = RSI_indica(open_BTC['Open'])
ETH_rsi, ETH_ema_up, ETH_ema_down = RSI_indica(open_ETH['Open'])
BNB_rsi, BNB_ema_up, BNB_ema_down = RSI_indica(open_BNB['Open'])

def graph_RSI (rsi, ema_up, ema_down):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=rsi.index, y=rsi, mode='lines', name='RSI', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=ema_up.index, y=ema_up, mode='lines', name='EMA Up', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=ema_down.index, y=ema_down, mode='lines', name='EMA Down', line=dict(color='red')))
    fig.update_layout(
        title="RSI et ses composantes (EMA Up et EMA Down)",
        xaxis_title="Périodes",
        yaxis_title="Valeur",
        template="plotly_dark",  
        showlegend=True
    )

    fig.show()

graph_rsi_BTC =graph_RSI(BTC_rsi,BTC_ema_up,BTC_ema_down)
graph_rsi_ETH =graph_RSI(ETH_rsi,ETH_ema_up,ETH_ema_down)
graph_rsi_BNB =graph_RSI(BNB_rsi,BNB_ema_up,BNB_ema_down)

 

graph EMA

In [316]:
def graph_EMA(ema_up,ema_down):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ema_up.index, y=ema_up, mode='lines', name='EMA Up', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=ema_down.index, y=ema_down, mode='lines', name='EMA Down', line=dict(color='red')))
    fig.update_layout(
    title="EMA Up et EMA Down",
    xaxis_title="Périodes",
    yaxis_title="Valeur",
    template="plotly_dark",  
    showlegend=True
    )

    fig.show()
    return fig


graph_EMA(BTC_ema_up,BTC_ema_down)

VAR avec BTC come ref avec sktime

In [317]:
def fr(df):
    fig = go.Figure()
    
    #bougies OHLC
    fig.add_trace(go.Candlestick(
        x=df['Timestamp'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name="OHLC",
        increasing_line_color='green',
        decreasing_line_color= 'red',

        
    ))
    fig.show()

fr(df_OHLC_BTC)

Tableau positions conseillé

In [339]:
def prix(prevision):
    prix_entrée = round(min(prevision), 2)
    prix_cible = round(max(prevision), 2)
    prix_stop = round(prix_entrée * 0.99, 2)


    return prix_entrée, prix_cible, prix_stop

prix_conseil_BTC= prix(BTC_pred)
prix_conseil_ETH=prix(ETH_pred)
prix_conseil_BNB=prix(BNB_pred)


df_prix_conseil = pd.DataFrame({
    "BTC":{
        "prix_entrée":prix_conseil_BTC[0],
        "prix_cible" : prix_conseil_BTC[1],
        "prix_sortie" : prix_conseil_BTC[2],

    },
    "ETH":{
        "prix_entrée":prix_conseil_ETH[0],
        "prix_cible" : prix_conseil_ETH[1],
        "prix_sortie" : prix_conseil_ETH[2],       
    } ,
    "BNB":{
        "prix_entrée":prix_conseil_BNB[0],
        "prix_cible" : prix_conseil_BNB[1],
        "prix_sortie" : prix_conseil_BNB[2],       
    } 
})

# Afficher le DataFrame
df_prix_conseil


,BTC,ETH,BNB
prix_entrée,90761.38,3170.43,628.36
prix_cible,91006.01,3198.55,632.31
prix_sortie,89853.77,3138.73,622.08


Historique prix 

#importation données 

In [319]:
from binance.client import Client
import pandas as pd

client = Client() #connection API public Binance 


symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']
start_date = "1 year ago UTC"  # Période de 1 an

# Fonction pour obtenir des données OHLC
def get_ohlc_data(symbol, interval):
    klines = client.get_historical_klines(symbol, interval, start_date)
    # Transformation des données en DataFrame
    data = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    
    

data_monthly = {symbol: get_ohlc_data(symbol, Client.KLINE_INTERVAL_1MONTH) for symbol in symbols}
data_weekly = {symbol: get_ohlc_data(symbol, Client.KLINE_INTERVAL_1WEEK) for symbol in symbols}
data_daily= {symbol: get_ohlc_data(symbol, Client.KLINE_INTERVAL_3DAY ) for symbol in symbols}



In [320]:
data_monthly['BNBUSDT']

,timestamp,open,high,low,close,volume
0,2023-12-01,227.60000000,338.30000000,224.80000000,311.80000000,31559599.09200000
1,2024-01-01,311.90000000,334.30000000,287.50000000,300.50000000,26121085.47600000
2,2024-02-01,300.50000000,427.30000000,296.40000000,399.40000000,21405632.33800000
3,2024-03-01,399.30000000,645.20000000,356.10000000,606.70000000,41560483.98500000
4,2024-04-01,606.70000000,634.50000000,508.00000000,578.40000000,18881921.68900000
5,2024-05-01,578.40000000,631.40000000,536.70000000,593.70000000,10105224.99200000
6,2024-06-01,593.80000000,721.80000000,551.20000000,582.20000000,9602003.78070000
7,2024-07-01,582.30000000,605.60000000,454.80000000,576.50000000,8036736.77500000
8,2024-08-01,576.50000000,600.60000000,400.00000000,532.90000000,10004287.11700000
9,2024-09-01,532.90000000,618.60000000,471.30000000,567.40000000,7343488.67100000


In [321]:
def graph_OHLC(df, interval, symbol):

    fig = go.Figure()
    
    #bougies OHLC
    fig.add_trace(go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name="OHLC",
        increasing_line_color='green',
        decreasing_line_color= 'red',

        
    ))
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        title="Historique" + interval + " des prix" + symbol ,
        xaxis_title="Dates",
        yaxis_title="Prix en USDT"
        )
    

#historique BTC
hist_BTC_3d= graph_OHLC(data_daily['BTCUSDT']," journalier ", " du BTC")
hist_BTC_sem=graph_OHLC(data_weekly['BTCUSDT'],"hebdomadaire", " du BTC")
hist_BTC_m=graph_OHLC(data_monthly['BTCUSDT']," mensuel ", " du BTC")

#ETH
hist_ETH_3d=graph_OHLC(data_daily['ETHUSDT']," journalier ", " de l'ETH")
hist_ETH_sem=graph_OHLC(data_weekly['ETHUSDT'],"hebdomadaire", " de l'ETH")
hist_ETH_m=graph_OHLC(data_monthly['ETHUSDT']," mensuel ", " de l'ETH")

#BNB

hist_BNB_3d=graph_OHLC(data_daily['BNBUSDT']," journalier ", " du BNB")
hist_BNB_sem=graph_OHLC(data_weekly['BNBUSDT'],"hebdomadaire", " du BNB")
hist_BNB_m=graph_OHLC(data_monthly['BNBUSDT']," mensuel ", " du BNB")




In [322]:
data_daily['BTCUSDT']['volume']/1000

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [334]:



def graph_hist_vol(df, interval, symbol,dtick_val):

    df['volume']=pd.to_numeric(df['volume'], errors='coerce')
    df['volume_conv']=(df['volume']/100).round() # nouvelle col pour éviter les probleme de perfs
    fig = go.Figure()

    # Ajouter un histogramme pour le volume
    fig.add_trace(go.Bar(
        x=df['timestamp'], 
        y=df['volume_conv'],
        name='Volume',
        marker=dict(color='rgb(0, 0, 255, 255)')
    ))
    
    # Mise en forme du graphique
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        title=f"Historique {interval} des volumes {symbol}",
        xaxis_title="Dates",
        yaxis_title="Volume (en milliers)",
        yaxis=dict(
            tickmode="linear",         
            dtick=dtick_val)
    )
    fig.show()
    
    

#BTC
#vol_BTC_D=graph_hist_vol(data_daily['BTCUSDT'], 'daily', 'BTC',20)
vol_BTC_M=graph_hist_vol(data_monthly['BTCUSDT'], 'mensuel', 'BTC',50)
vol_BTC_W=graph_hist_vol(data_weekly['BTCUSDT'], 'hebdomadaire', 'BTC',10)

#ETH

#vol_ETH_D=graph_hist_vol(data_daily['ETHUSDT'], 'daily', 'ETH',20)
vol_ETH_M=graph_hist_vol(data_monthly['ETHUSDT'], 'mensuel', 'ETH',250)
vol_ETH_W=graph_hist_vol(data_weekly['ETHUSDT'], 'hebdomadaire', 'ETH',100)

#BNB

#vol_BNB_D=graph_hist_vol(data_daily['BNBUSDT'], 'daily', 'BNB',20)
vol_BNB_M=graph_hist_vol(data_monthly['BNBUSDT'], 'mensuel', 'BNB',1000)
vol_BNB_W=graph_hist_vol(data_weekly['BNBUSDT'], 'hebdomadaire', 'BNB',100)
    

In [281]:
print(es)

0       766.0
1      1287.0
2       799.0
3       888.0
4      1012.0
        ...  
117     653.0
118    1824.0
119    1153.0
120    2664.0
121     875.0
Name: volume, Length: 122, dtype: float64


In [338]:
def graph_hist_vo_2(df, interval, symbol,dtick_val):
    fig = go.Figure()
    
    # Ajouter un histogramme pour le volume
    fig.add_trace(go.bar(
        x=df['timestamp'], 
        y=df['volume'],
        name='Volume',
        marker=dict(color='rgb(0, 0, 255, 255)'),
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        title=f"Historique {interval} des volumes {symbol}",
        xaxis_title="Dates",
        yaxis_title="Volume",
        yaxis=dict(
            dtick=dtick_val)

    ))
    
    fig.show()

vol_BNB_W2=graph_hist_vol(data_weekly['BNBUSDT'], 'hebdomadaire', 'BNB',100)

API tendace 


In [258]:
import requests


url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",  
    "per_page": 10,  # nbr de cryptos par page classé par rank market cap
    "page": 1,  # nbr de page  
}

response = requests.get(url, params=params)
response.raise_for_status()
classement = response.json()








In [249]:
df_gagnant.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

In [261]:
df_classement= pd.DataFrame(classement,columns=["name","price_change_percentage_24h"])
df_classement.sort_values(by="price_change_percentage_24h",ascending= False)

,name,price_change_percentage_24h
9,Cardano,17.60055
6,XRP,11.44618
0,Bitcoin,4.00124
3,Solana,2.57307
2,Tether,-0.03476
7,USDC,-0.07087
8,Lido Staked Ether,-0.71904
1,Ethereum,-0.76615
5,Dogecoin,-2.56810
4,BNB,-2.73730


File actu

In [266]:
import requests


url = "https://cryptopanic.com/api/v1/posts/"

params = {
    "auth_token": "426c945ba63a8df1f3957d45c79b8e6393cc8e67",  # clé API
    "filter": "trending",  
}
response = requests.get(url, params=params)
response.raise_for_status()
news = response.json()





,previous,results
0,NaN,"{'kind': 'news', 'domain': 'dailycoin.com', 'v..."
1,NaN,"{'kind': 'news', 'domain': 'u.today', 'votes':..."
2,NaN,"{'kind': 'news', 'domain': 'decrypt.co', 'vote..."
3,NaN,"{'kind': 'news', 'domain': '@DaanCrypto', 'vot..."
4,NaN,"{'kind': 'news', 'domain': 'coinpaprika.com', ..."
5,NaN,"{'kind': 'news', 'domain': 'coinpaprika.com', ..."
6,NaN,"{'kind': 'news', 'domain': 'beincrypto.com', '..."
7,NaN,"{'kind': 'reddit', 'domain': 'reddit.com', 'vo..."
8,NaN,"{'kind': 'news', 'domain': 'cryptopolitan.com'..."
9,NaN,"{'kind': 'news', 'domain': '@BTC_Archive', 'vo..."
